In [11]:
!pip install surprise

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from surprise import Dataset, Reader, SVD, SVDpp, NMF
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split
import random

In [13]:
# цей код — приклад того, як можна обчислити алгоритм для рекомендацій (приклад, що показує, як можна завантажити
# набір даних, розділити його для 5-кратної перехресної перевірки та обчислити MAE та RMSE алгоритму SVD.)
# Він використовує дані про фільми та оцінки, щоб створити модель, яка передбачатиме, які фільми можуть сподобатися користувачам.
# Потім модель перевіряється, щоб переконатися, що вона працює правильно.

data = Dataset.load_builtin('ml-100k')

# Використовуйте алгоритм SVD
# algo = SVD(): Використовує алгоритм SVD, щоб створити модель, яка передбачатиме, які фільми сподобаються користувачам.

# Виконати 5-кратну перехресну перевірку та роздрукувати результат
# Перевіряє, наскільки добре працює модель, використовуючи метод під назвою "крос-валідація".
# Вона ділить дані на 5 частин і перевіряє, наскільки точно модель передбачає оцінки фільмів. Потім виводить результати.
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [14]:
# Бере дані про те, які користувачі ставили якісь оцінки (рейтинг) різним речам (наприклад, фільмам).
raw_ratings = data.raw_ratings

# Створює таблицю (DataFrame) з цих даних і дає імена стовпцям: "user" (користувач),
# "item" (предмет, наприклад, фільм), "rating" (оцінка) та "timestamp" (час, коли була поставлена ​​оцінка) .
df = pd.DataFrame(raw_ratings, columns=['user', 'item', 'rating', 'timestamp'])
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [15]:
# Цей код допомагає знайти найкращі параметри для моделі, щоб вона давала точні передбачення.
# param_grid: вибираємо різні параметри (Ці параметри ми вибираємо самостійно.
# Можна змінювати для найкращих результатів), які хочемо протестувати, щоб зрозуміти, які дадуть кращий результат.
# Наприклад, скільки разів навчати модель (n_epochs), як швидко вона навчається (lr_all),
# і наскільки ми хочемо уникнути перенавчання (reg_all).
param_grid = {
    'n_epochs': [12, 24],
    'lr_all': [0.01, 0.03],
    'reg_all': [0.1, 0.2]
}

# GridSearchCV: Ця команда пробує всі можливі комбінації параметрів з param_grid і вибирає найкращі.
# Вона використовує метод крос-валідації (cv=3), щоб перевірити, які параметри найкраще підходять для передбачення.
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

# Тут програма навчає модель даних з різними комбінаціями параметрів і вибирає найкращу.
gs.fit(data)

# Виводить на екран найкращий результат (RMSE) та параметри, що призвели до цього результату.
print("Best RMSE score:", gs.best_score['rmse'])
print("Best parameters:", gs.best_params['rmse'])

Best RMSE score: 0.9231669463273461
Best parameters: {'n_epochs': 24, 'lr_all': 0.01, 'reg_all': 0.1}


In [16]:
# код оцінює, наскільки хороша модель, використовуючи кращі параметри
# створюємо модель SVD і передаємо їй найкращі параметри, які знайшли раніше
# (скільки разів навчати модель, швидкість навчання та рівень регуляризації).
algo = SVD(n_epochs=gs.best_params['rmse']['n_epochs'],
           lr_all=gs.best_params['rmse']['lr_all'],
           reg_all=gs.best_params['rmse']['reg_all'])

# cross_validate – перевіряємо, наскільки добре наша модель працює. Крос-валідація поділяє дані на 3 частини (cv=3),
# щоб навчати модель на одній частині і перевіряти на іншій. Це допомагає зрозуміти наскільки модель точна.
results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
print("\nResults with the best parameters:")

# середні результати після всіх перевірок. RMSE і MAE показують, наскільки добре модель передбачає оцінки
print("RMSE: ", np.mean(results['test_rmse']))
print("MAE: ", np.mean(results['test_mae']))

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9264  0.9252  0.9229  0.9248  0.0014  
MAE (testset)     0.7335  0.7311  0.7301  0.7316  0.0014  
Fit time          2.05    3.40    4.39    3.28    0.96    
Test time         0.43    0.63    0.60    0.55    0.09    

Results with the best parameters:
RMSE:  0.924813340549017
MAE:  0.7315617541188404


In [17]:
# код допомагає підібрати найкращі параметри для моделі NMF, щоб вона точно передбачала результати.
from surprise import NMF

# вибираємо параметри, які хочемо протестувати моделі NMF. Ці параметри впливають на те, як модель
# навчається і наскільки добре вона передбачає.
# Наприклад, скільки разів навчати модель (n_epochs), швидкість навчання для користувачів
# та фільмів (lr_bu, lr_bi), і як сильно штрафувати модель за складність (reg_pu, reg_qi).
param_grid = {
    'n_epochs': [50, 100],
    'lr_bu': [0.002, 0.005],
    'lr_bi': [0.002, 0.005],
    'reg_pu': [0.06, 0.1],
    'reg_qi': [0.06, 0.1]
}

# GridSearchCV: Ця команда пробує різні комбінації параметрів з param_grid і вибирає ті,
# які дають найкращий результат.
# Вона використовує крос-валідацію (cv=3), щоб перевірити кожну комбінацію.
gs = GridSearchCV(NMF, param_grid, measures=['rmse', 'mae'], cv=3)

# Програма навчає модель NMF з різними комбінаціями параметрів і знаходить найкращі.
gs.fit(data)

In [18]:
# код допомагає оцінити, наскільки добре працює модель NMF з підібраними параметрами
# Виводить на екран найкращий результат RMSE (помилка) для моделі NMF та параметри, які дали цей результат.
print("Best RMSE score for NMF:", gs.best_score['rmse'])
print("Best parameters for NMF:", gs.best_params['rmse'])

# Ініціалізація алгоритму з найкращими параметрами
best_params = gs.best_params['rmse']

# Ми створюємо модель NMF з найкращими параметрами, які знайшли раніше
algo_nmf = NMF(n_epochs=best_params['n_epochs'],
               lr_bu=best_params['lr_bu'],
               lr_bi=best_params['lr_bi'],
               reg_pu=best_params['reg_pu'],
               reg_qi=best_params['reg_qi'])

# cross_validate – перевіряємо, наскільки добре працює наша модель NMF. Використовуємо
# крос-валідацію, яка поділяє дані на 3 частини, щоб переконатись, що модель працює точно.
results_nmf = cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

# Виводить середні значення RMSE та MAE, які показують, наскільки добре модель передбачає оцінки.
print("\nResults with the best parameters for NMF:")
print("RMSE: ", np.mean(results_nmf['test_rmse']))
print("MAE: ", np.mean(results_nmf['test_mae']))

Best RMSE score for NMF: 0.9440994493382479
Best parameters for NMF: {'n_epochs': 100, 'lr_bu': 0.005, 'lr_bi': 0.005, 'reg_pu': 0.1, 'reg_qi': 0.1}
Evaluating RMSE, MAE of algorithm NMF on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9503  0.9479  0.9478  0.9486  0.0012  
MAE (testset)     0.7544  0.7504  0.7495  0.7514  0.0021  
Fit time          3.94    7.68    4.37    5.33    1.67    
Test time         0.37    0.50    0.37    0.42    0.06    

Results with the best parameters for NMF:
RMSE:  0.948625874329696
MAE:  0.7514351404938409


In [19]:
# код допомагає знайти найкращі параметри для моделі SVD++, щоб вона краще передбачала результати.

from surprise import SVDpp
from surprise.model_selection import GridSearchCV, cross_validate
import numpy as np

# визначаємо параметри, які тестуватимемо для моделі SVD++. Наприклад, скільки разів навчати
# модель (n_epochs), швидкість навчання (lr_all) і як сильно штрафувати модель за складність (reg_all).
param_grid = {
    'n_epochs': [20, 50],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.1]
}

# GridSearchCV: Ця команда тестує всі комбінації параметрів param_grid і вибирає ті, які дають
# найкращий результат. Крос-валідація (CV=3) допомагає перевірити точність кожної комбінації.
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)

# Програма навчає модель SVD++ з різними комбінаціями параметрів і знаходить найкращі з них.
gs.fit(data)

# Виводить на екран найкращий результат RMSE (помилка) та параметри, що призвели до цього результату.
print("Best RMSE score for SVD++:", gs.best_score['rmse'])
print("Best parameters for SVD++:", gs.best_params['rmse'])

Best RMSE score for SVD++: 0.9220133437157426
Best parameters for SVD++: {'n_epochs': 50, 'lr_all': 0.005, 'reg_all': 0.1}


In [20]:
# Ми беремо найкращі параметри, які знайшли раніше.
best_params = gs.best_params['rmse']
# Створюємо модель SVD++ з цими найкращими параметрами
algo_svdpp = SVDpp(n_epochs=best_params['n_epochs'],
                   lr_all=best_params['lr_all'],
                   reg_all=best_params['reg_all'])

# cross_validate: Перевіряємо, як добре працює наша модель SVD++ за допомогою крос-валідації. Це робиться для того,
# щоб побачити, наскільки точно модель передбачає результати.
results_svdpp = cross_validate(algo_svdpp, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

# Виводимо середні значення RMSE (помилка) та MAE (середня абсолютна помилка), щоб побачити, як добре працює модель з вибраними параметрами.
print("\nResults with the best parameters for SVD++:")
print("RMSE: ", np.mean(results_svdpp['test_rmse']))
print("MAE: ", np.mean(results_svdpp['test_mae']))

Evaluating RMSE, MAE of algorithm SVDpp on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9187  0.9212  0.9206  0.9202  0.0011  
MAE (testset)     0.7260  0.7283  0.7266  0.7270  0.0010  
Fit time          48.75   47.69   49.01   48.48   0.57    
Test time         8.08    8.22    7.98    8.09    0.10    

Results with the best parameters for SVD++:
RMSE:  0.9201633638452135
MAE:  0.7269843518017197


Найкращий результат по RMSE:

SVD++: 0.9220

SVD: 0.9232

NMF: 0.9441

Таким чином, модель SVD++ має найменшу помилку (RMSE), що робить її найкращою з представлених моделей.